# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [135]:
# all import statements needed for the project, for example:

import math
import os

import bs4
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd
from geopy.distance import distance

In [ ]:
"""any constants you might need; some have been added for you, and some you need to fill in"""

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = ""
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
"""Make sure the QUERY_DIRECTORY exists"""
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Load Taxi Zones

In [23]:
""""""
def load_taxi_zones(shapefile):
    gdf = gpd.read_file(shapefile)

    taxi_zones = []

    for index, row in gdf.iterrows():
        zone = row.iloc[3]
        locationId = row.iloc[4]
        geometry = row.iloc[6]
        
        row_object = { "zone": zone, "locationId": locationId, "geometry": geometry }
        taxi_zones.append(row_object)
    
    return taxi_zones

In [31]:
""""""
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    for i in loaded_taxi_zones:
        if i['locationId'] == zone_loc_id:
            return i['geometry']
        

# zones = load_taxi_zones('taxi_zones.shp')
# lookup_coords_for_taxi_zone_id(3, zones)

### Calculate distance

In [1]:
""""""
def calculate_distance_with_coords(from_coord, to_coord):
    pickup_latitude, pickup_longitude = from_coord
    dropoff_latitude, dropoff_longitude = to_coord
    return distance((pickup_latitude, pickup_longitude), (dropoff_latitude, dropoff_longitude)).miles

In [4]:
"""test - calculate_distance_with_coords()"""
from_coord = (40.7128, -74.0060)  # New York City
to_coord = (34.0522, -118.2437)  # Los Angeles
assert round(calculate_distance_with_coords(from_coord, to_coord), 2) == 2449.53

AssertionError: 

In [ ]:
def calculate_distance_with_zones(from_zone, to_zone):
    raise NotImplementedError()

In [5]:
"""This fucntion adds a new column with the distnace between coordinates to the Dataframe.
 The input is a dataframe and the output is the new dataframe"""
def add_distance_column(dataframe):
    # Apply the calculate_distance_with_coords function to each row of the DataFrame
    distances = dataframe.apply(lambda row: calculate_distance_with_coords(
        (row["pickup_latitude"], row["pickup_longitude"]),
        (row["dropoff_latitude"], row["dropoff_longitude"])
    ), axis=1)
    
    # Add the distances as a new column to the DataFrame
    dataframe["distance"] = distances
    
    return dataframe

In [6]:
""" test - add_distance_column()"""
data = {
    "pickup_latitude": [40.7128, 41.8781],
    "pickup_longitude": [-74.0060, -87.6298],
    "dropoff_latitude": [34.0522, 37.7749],
    "dropoff_longitude": [-118.2437, -122.4194]
}
df = pd.DataFrame(data)

expected_distances = [
    distance((40.7128, -74.0060), (34.0522, -118.2437)).miles,
    distance((41.8781, -87.6298), (37.7749, -122.4194)).miles
]

df_with_distance = add_distance_column(df)
assert all(df_with_distance["distance"].round(2) == expected_distances)

AssertionError: 

### Process Taxi Data

In [ ]:
""""""
def download_files(month, year):
    formatted_month = f"{month:02d}"
    current_dir = os.getcwd()
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{formatted_month}.parquet"

    response = requests.get(url, stream=True)
    with open(f"{current_dir}\yellow_taxi_{year}_{formatted_month}.parquet", "wb") as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)

years = list(range(2009, 2016))
months = list(range(1, 13))

for year in years:
    if year < 2015:
        for month in months:
            download_files(month, year)
    else:
        for month in range(1, 7):
            download_files(month, year)

In [71]:
""""""
def get_all_urls_from_taxi_page(taxi_page):
    try:
        response = requests.get(taxi_page)

        soup = bs4.BeautifulSoup(response.content, 'html.parser')
        urls = []

        for link in soup.find_all('a'):
            href = link.get('href')
            if href is not None:
                urls.append(href)

        return urls
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
# get_all_urls_from_taxi_page("https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page")

In [70]:
""""""
def filter_taxi_parquet_urls(all_urls):
    parquet_urls = []

    if all_urls is not None:
        for i in all_urls:
            if i.endswith(".parquet"):
                parquet_urls.append(i)
    return parquet_urls

# allUrlsData = get_all_urls_from_taxi_page("https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page")
# filter_taxi_parquet_urls(allUrlsData)

['http://www1.nyc.gov', '/311/index.page', '/home/search/index.page', '#', '/site/tlc/index.page', 'http://www1.nyc.gov/home/text-size.page', '#', '/site/tlc/index.page', '/site/tlc/about/about-tlc.page', '/site/tlc/passengers/your-ride.page', '/site/tlc/drivers/get-a-drivers-license.page', '/site/tlc/vehicles/get-a-vehicle-license.page', '/site/tlc/businesses/yellow-cab.page', '/site/tlc/tlc-online/tlc-online.page', '/site/tlc/about/about-tlc.page', '/site/tlc/about/data-and-research.page', '/site/tlc/about/tlc-initiatives.page', '/site/tlc/about/contact-tlc.page', '/site/tlc/about/data.page', '/site/tlc/about/pilot-programs.page', '/site/tlc/about/industry-reports.page', '/site/tlc/about/tlc-trip-record-data.page', '/site/tlc/about/request-data.page', '#', '#', '#', '#', '#', 'javascript:expandAll();', 'javascript:collapseAll();', 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet', 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.

In [ ]:
def get_and_clean_month(url):
    raise NotImplementedError()

In [ ]:
""""""
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

In [ ]:
""""""
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = find_taxi_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [ ]:
taxi_data = get_taxi_data()

In [ ]:
taxi_data.head()

### Processing Uber Data

In [167]:
"""This function first loads the uber data from the csv file. 
We then filter based on coordinates to make sure the rides are within the coordinates we want.
We also remove trips with 0 passangers and no fares. We further remove trips with passangers above 6 as that 
is uber policy. Lastly we remove trips with no distace between dropoff and pickup. The output is the
cleaned dataframe"""

def load_and_clean_uber_data(csv_file):

    # Reading in file into a data frame 
    uber_data = pd.read_csv(csv_file)

    # Filter data based on pickup and dropoff latitude/longitude(40.560445, -74.242330) and (40.908524, -73.717047).

    uber_data = uber_data[(uber_data["pickup_latitude"] >= 40.560445) & 
                      (uber_data["pickup_longitude"] >= -74.242330) & 
                      (uber_data["pickup_latitude"] <= 40.908524) & 
                      (uber_data["pickup_longitude"] <= -73.717047) &
                      (uber_data["dropoff_latitude"] >= 40.560445) & 
                      (uber_data["dropoff_longitude"] >= -74.242330) & 
                      (uber_data["dropoff_latitude"] <= 40.908524) & 
                      (uber_data["dropoff_longitude"] <= -73.717047)]
    
    # Checking if there are any null values for pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
    null_drop_lat = uber_data[uber_data['dropoff_latitude'].isnull()]
    null_drop_long = uber_data[uber_data['dropoff_longitude'].isnull()]
    null_pick_lat= uber_data[uber_data['pickup_latitude'].isnull()]
    null_pick_long = uber_data[uber_data['pickup_longitude'].isnull()]

    # Return True, if none of the colums have null values 

   # if null_drop_lat.empty & null_drop_long.empty & null_pick_lat.empty & null_pick_long.empty :
        #print(True)
    #else:
       # print(False)
    
    # Removing rows where passamger count is 0 
    uber_data = uber_data[uber_data['passenger_count']!=0]

    # Removing rows where fare is 0 
    uber_data = uber_data[uber_data['fare_amount']!=0]


    # Removing rows with passanger data is abnormally large 
    uber_data = uber_data[uber_data['passenger_count']<=6]

    # Checking datatypes for all columns 
    #print(uber_data.dtypes)

    #Making sure pickup time is a datetime object and normalizing the name 
    uber_data ['pickup_time'] = pd.to_datetime(uber_data ['pickup_datetime'])
 


    return uber_data


In [168]:
load_and_clean_uber_data("uber_rides_sample.csv")

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_time
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07 19:52:06+00:00
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17 20:04:56+00:00
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24 21:45:00+00:00
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,2009-06-26 08:22:21+00:00
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,2014-08-28 17:47:00+00:00
...,...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1,2012-10-28 10:49:00+00:00
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1,2014-03-14 01:09:00+00:00
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2,2009-06-29 00:42:00+00:00
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1,2015-05-20 14:56:25+00:00


In [169]:
""" We use the add distance column fcuntion we had defined before to add a new column with the distance 
of the ride to our uber data. We also drop columns where the distance of the ride is ==0"""

def get_uber_data():
    uber_dataframe = load_and_clean_uber_data("uber_rides_sample.csv")
    add_distance_column(uber_dataframe)
    uber_dataframe = uber_dataframe.drop(index=uber_dataframe[uber_dataframe['distance'] == 0].index)
    return uber_dataframe


In [170]:
final_uber_data = get_uber_data()

In [164]:
#Removing unnecessary columns 
final_uber_data = final_uber_data.drop('Unnamed: 0', axis=1)
final_uber_data = final_uber_data.drop('key', axis=1)

In [171]:
final_uber_data

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_time,distance
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,2015-05-07 19:52:06+00:00,1.044594
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,2009-07-17 20:04:56+00:00,1.525071
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,2009-08-24 21:45:00+00:00,3.131464
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,2009-06-26 08:22:21+00:00,1.032372
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,2014-08-28 17:47:00+00:00,2.786061
...,...,...,...,...,...,...,...,...,...,...,...
199995,42598914,2012-10-28 10:49:00.00000053,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1,2012-10-28 10:49:00+00:00,0.069673
199996,16382965,2014-03-14 01:09:00.0000008,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1,2014-03-14 01:09:00+00:00,1.167951
199997,27804658,2009-06-29 00:42:00.00000078,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2,2009-06-29 00:42:00+00:00,7.995752
199998,20259894,2015-05-20 14:56:25.0000004,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1,2015-05-20 14:56:25+00:00,2.197512


### Processing Weather Data

In [172]:
"""This function takes all the weather files, iterates through them and merges them 
into one dataframe. The output is the combined dataframe"""

def get_all_weather_csvs():
    years = list(range(2009, 2016))

    # Initialize an empty list to store the dataframes
    dataframes = []

    # Iterate over the weather files
    for year in years:
        filepath = f"{year}_weather.csv"
        df = pd.read_csv(filepath)
        dataframes.append(df)

    # Concatenate all the dataframes into a single dataframe
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

get_all_weather_csvs()

/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (17,78) have mix

,STATION,DATE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,SOURCE,HourlyAltimeterSetting,HourlyDewPointTemperature,...,BackupDirection,BackupDistance,BackupDistanceUnit,BackupElements,BackupElevation,BackupEquipment,BackupLatitude,BackupLongitude,BackupName,WindEquipmentChangeDate
0,72505394728,2009-01-01T00:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.01,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
1,72505394728,2009-01-01T01:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.03,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
2,72505394728,2009-01-01T02:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.07,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
3,72505394728,2009-01-01T03:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.09,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
4,72505394728,2009-01-01T04:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",AUTO,4,30.12,3.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77967,72505394728,2015-12-31T21:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,6,30.12,26.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77968,72505394728,2015-12-31T22:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,7,30.11,28.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77969,72505394728,2015-12-31T23:51:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",FM-15,7,30.1,26.0,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18
77970,72505394728,2015-12-31T23:59:00,40.77898,-73.96925,42.7,"NY CITY CENTRAL PARK, NY US",SOD,6,NaN,NaN,...,ESE,0.5,mi,SNOW,NaN,SNOWBOARD,NaN,NaN,CENTRAL PARK ZOO,2006-09-18


In [173]:
df = get_all_weather_csvs()

df1 = df[['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'NAME','HourlyPrecipitation','HourlyWindGustSpeed', 'HourlyWindSpeed', 'DailyAverageWindSpeed','DailyPrecipitation']]

print(df1)

/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (8,9,10,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (7,8,9,10,17,18,42,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)
/var/folders/4y/mgwhf9fd22746wblcfjds5gc0000gn/T/ipykernel_21281/1746399148.py:13: DtypeWarning: Columns (17,78) have mix

           STATION                 DATE  LATITUDE  LONGITUDE  \
0      72505394728  2009-01-01T00:51:00  40.77898  -73.96925   
1      72505394728  2009-01-01T01:51:00  40.77898  -73.96925   
2      72505394728  2009-01-01T02:51:00  40.77898  -73.96925   
3      72505394728  2009-01-01T03:51:00  40.77898  -73.96925   
4      72505394728  2009-01-01T04:51:00  40.77898  -73.96925   
...            ...                  ...       ...        ...   
77967  72505394728  2015-12-31T21:51:00  40.77898  -73.96925   
77968  72505394728  2015-12-31T22:51:00  40.77898  -73.96925   
77969  72505394728  2015-12-31T23:51:00  40.77898  -73.96925   
77970  72505394728  2015-12-31T23:59:00  40.77898  -73.96925   
77971  72505394728  2015-12-31T23:59:00  40.77898  -73.96925   

                              NAME HourlyPrecipitation  HourlyWindGustSpeed  \
0      NY CITY CENTRAL PARK, NY US                 NaN                 37.0   
1      NY CITY CENTRAL PARK, NY US                 NaN                 28

In [174]:
df2 = df1.dropna(subset=['HourlyPrecipitation', 'HourlyWindGustSpeed'])

column_types = df2.dtypes

print(column_types)

# we see that the averages for wind speed and precipitation are null for all values so we can drop the columns 

# We also doing need the hourly wind gust speed as we will be using the hourly wind speed, we can drop that column as well

df2 = df2.drop(columns=['DailyAverageWindSpeed','DailyPrecipitation', 'HourlyWindGustSpeed','LATITUDE', 'LONGITUDE'])
df2['DATE'] = pd.to_datetime(df['DATE'])

df2

# Removing all rows where Hourly preicipitation has the value "T" as we do not need to measure trace amounts 

df3 = df2[df2['HourlyPrecipitation'] != "T"]
df3





STATION                    int64
DATE                      object
LATITUDE                 float64
LONGITUDE                float64
NAME                      object
HourlyPrecipitation       object
HourlyWindGustSpeed      float64
HourlyWindSpeed          float64
DailyAverageWindSpeed    float64
DailyPrecipitation        object
dtype: object


,STATION,DATE,NAME,HourlyPrecipitation,HourlyWindSpeed
154,72505394728,2009-01-06 20:00:00,"NY CITY CENTRAL PARK, NY US",0.01,10.0
161,72505394728,2009-01-06 23:38:00,"NY CITY CENTRAL PARK, NY US",0.02,11.0
168,72505394728,2009-01-07 02:51:00,"NY CITY CENTRAL PARK, NY US",0.09,13.0
169,72505394728,2009-01-07 03:51:00,"NY CITY CENTRAL PARK, NY US",0.06,15.0
170,72505394728,2009-01-07 04:51:00,"NY CITY CENTRAL PARK, NY US",0.07,16.0
...,...,...,...,...,...
77875,72505394728,2015-12-29 10:51:00,"NY CITY CENTRAL PARK, NY US",0.02,10.0
77877,72505394728,2015-12-29 11:33:00,"NY CITY CENTRAL PARK, NY US",0.02,8.0
77878,72505394728,2015-12-29 11:51:00,"NY CITY CENTRAL PARK, NY US",0.02,6.0
77957,72505394728,2015-12-31 11:51:00,"NY CITY CENTRAL PARK, NY US",0.00,9.0


In [134]:

df3
df4 = df3.drop(columns=["STATION"])

df4 = df4.reset_index()


df4['DATE'] = df4['DATE'].apply(lambda x: x.to_pydatetime())

# Print the DataFrame
df4
df4.dtypes

df4['DATE'] = pd.to_datetime(df4['DATE'])

df4
df4.dtypes



df4['HourlyPrecipitation'] = df4['HourlyPrecipitation'].str.replace(r'(\d+)\s*[sS]$', r'\1', regex=True)

# convert column "A" from object to float
df4['HourlyPrecipitation'] = df4['HourlyPrecipitation'].astype(float)

grouped_sum = df4.groupby([df4['DATE'].dt.year, df4['DATE'].dt.dayofyear, df4['DATE'].dt.hour]).sum()[['HourlyPrecipitation', "HourlyWindSpeed" ]]
grouped_sum










HourlyPrecipitation  HourlyWindSpeed
DATE DATE DATE                                      
2009 6    20                   0.01             10.0
          23                   0.02             11.0
     7    2                    0.09             13.0
          3                    0.06             15.0
          4                    0.07             16.0
...                             ...              ...
2015 363  9                    0.07             19.0
          10                   0.13             29.0
          11                   0.04             14.0
     365  11                   0.00              9.0
          20                   0.00             10.0

[5814 rows x 2 columns]

AttributeError: 'function' object has no attribute 'reset_index'

In [ ]:
"""Roll up the data to daily"""
def clean_month_weather_data_daily(csv_file):
    raise NotImplementedError()

In [77]:

def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs()
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [78]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [73]:
hourly_weather_data.head()

In [ ]:
daily_weather_data.head()

## Part 2: Storing Cleaned Data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)